In [9]:
import pandas as pd
from sklearn import *
import numpy as np

In [10]:
df = pd.read_csv("/data/kaggle/data_combined_cleaned.csv")
df = df.drop(columns="Id")
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalesPrice
0,60,RL,65.0,8450,Pave,None,Reg,Lvl,Inside,Gtl,...,0,None,None,None,0,2,2008,WD,Normal,208500.0
1,20,RL,80.0,9600,Pave,None,Reg,Lvl,FR2,Gtl,...,0,None,None,None,0,5,2007,WD,Normal,181500.0
2,60,RL,68.0,11250,Pave,None,IR1,Lvl,Inside,Gtl,...,0,None,None,None,0,9,2008,WD,Normal,223500.0
3,70,RL,60.0,9550,Pave,None,IR1,Lvl,Corner,Gtl,...,0,None,None,None,0,2,2006,WD,Abnorml,140000.0
4,60,RL,84.0,14260,Pave,None,IR1,Lvl,FR2,Gtl,...,0,None,None,None,0,12,2008,WD,Normal,250000.0


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 79 columns):
MSSubClass       2919 non-null int64
MSZoning         2919 non-null object
LotFrontage      2919 non-null float64
LotArea          2919 non-null int64
Street           2919 non-null object
Alley            2919 non-null object
LotShape         2919 non-null object
LandContour      2919 non-null object
LotConfig        2919 non-null object
LandSlope        2919 non-null object
Neighborhood     2919 non-null object
Condition1       2919 non-null object
Condition2       2919 non-null object
BldgType         2919 non-null object
HouseStyle       2919 non-null object
OverallQual      2919 non-null int64
OverallCond      2919 non-null int64
YearBuilt        2919 non-null int64
YearRemodAdd     2919 non-null int64
RoofStyle        2919 non-null object
RoofMatl         2919 non-null object
Exterior1st      2919 non-null object
Exterior2nd      2919 non-null object
MasVnrType       2919 no

In [12]:
df = df.dropna()

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 0 to 1459
Data columns (total 79 columns):
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            1460 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-null object
Exterior2nd      1460 non-null object
MasVnrType       1460 no

In [15]:
# dependent variable/target varible/label
label = "SalesPrice"

# independepent variables/features/predictors
X = df.drop(columns=[label])


# vector for target variable
y = np.log(df[label])

# one hot encoding to conver categorical features into numeric
# drop_first: remove first value of a categorical feature ... it is redundant
X = pd.get_dummies(X, drop_first=True)

# Creating trainnig and test sets
# test_size is 30% of the whole
# random_state: to reprduce the same combination of training and test records
X_train, X_test, y_train, y_test = model_selection.train_test_split(X.values,                       
                                    y, test_size = 0.3, random_state = 1)


pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.LinearRegression())
])

pipe.fit(X_train, y_train)

# prediction on training and test data
y_train_pred = pipe.predict(X_train)
y_test_pred = pipe.predict(X_test)

print("R2 training:", metrics.r2_score(y_train, y_train_pred))
print("R2 test:", metrics.r2_score(y_test, y_test_pred))

print("Rmse training:", np.sqrt(metrics.mean_squared_error(y_train, y_train_pred)))
print("Rmse: testing", np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))

R2 training: 0.9494518297088844
R2 test: -3.0715943247509893e+22
Rmse training: 0.08606817747710854
Rmse: testing 76190488351.17424


In [20]:
est = pipe.steps[-1][-1]
weights = pd.DataFrame({"feature": X.columns, "coef": est.coef_})
weights["abs"] = np.abs(weights["coef"])
weights = weights.sort_values("abs", ascending = False)
weights

,feature,coef,abs
15,GrLivArea,1.501695e+11,1.501695e+11
13,2ndFlrSF,-1.233520e+11,1.233520e+11
12,1stFlrSF,-1.112376e+11,1.112376e+11
219,GarageFinish_None,-1.072428e+11,1.072428e+11
134,Exterior2nd_CBlock,8.844505e+10,8.844505e+10
120,Exterior1st_CBlock,-8.844505e+10,8.844505e+10
224,GarageQual_None,7.930425e+10,7.930425e+10
229,GarageCond_None,7.930320e+10,7.930320e+10
166,BsmtCond_None,6.149525e+10,6.149525e+10
218,GarageType_None,-5.136464e+10,5.136464e+10


In [24]:
# dependent variable/target varible/label
label = "SalesPrice"

# independepent variables/features/predictors
X = df.drop(columns=[label])


# vector for target variable
y = np.log(df[label])

# one hot encoding to conver categorical features into numeric
# drop_first: remove first value of a categorical feature ... it is redundant
X = pd.get_dummies(X, drop_first=True)

# Creating trainnig and test sets
# test_size is 30% of the whole
# random_state: to reprduce the same combination of training and test records
X_train, X_test, y_train, y_test = model_selection.train_test_split(X.values,                       
                                    y, test_size = 0.3, random_state = 1)


pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.Lasso(alpha=0.01))
])

pipe.fit(X_train, y_train)

# prediction on training and test data
y_train_pred = pipe.predict(X_train)
y_test_pred = pipe.predict(X_test)

print("R2 training:", metrics.r2_score(y_train, y_train_pred))
print("R2 test:", metrics.r2_score(y_test, y_test_pred))

print("Rmse training:", np.sqrt(metrics.mean_squared_error(y_train, y_train_pred)))
print("Rmse: testing", np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))

est = pipe.steps[-1][-1]
weights = pd.DataFrame({"feature": X.columns, "coef": est.coef_})
weights["abs"] = np.abs(weights["coef"])
weights = weights.sort_values("abs", ascending = False)
weights

R2 training: 0.8940480673803641
R2 test: 0.8785389987905549
Rmse training: 0.12460754170603186
Rmse: testing 0.15150859029885974


,feature,coef,abs
3,OverallQual,0.104135,0.104135
15,GrLivArea,0.095563,0.095563
25,GarageCars,0.055896,0.055896
5,YearBuilt,0.029465,0.029465
39,MSZoning_RM,-0.026432,0.026432
4,OverallCond,0.023309,0.023309
6,YearRemodAdd,0.021713,0.021713
70,Neighborhood_NridgHt,0.020148,0.020148
60,Neighborhood_Crawfor,0.019489,0.019489
16,BsmtFullBath,0.018030,0.018030


In [30]:
# dependent variable/target varible/label
label = "SalesPrice"

# independepent variables/features/predictors
X = df.drop(columns=[label])


# vector for target variable
y = np.log(df[label])

# one hot encoding to conver categorical features into numeric
# drop_first: remove first value of a categorical feature ... it is redundant
X = pd.get_dummies(X, drop_first=True)

# Creating trainnig and test sets
# test_size is 30% of the whole
# random_state: to reprduce the same combination of training and test records
X_train, X_test, y_train, y_test = model_selection.train_test_split(X.values,                       
                                    y, test_size = 0.3, random_state = 1)


pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.Ridge(alpha=20.))
])

pipe.fit(X_train, y_train)

# prediction on training and test data
y_train_pred = pipe.predict(X_train)
y_test_pred = pipe.predict(X_test)

print("R2 training:", metrics.r2_score(y_train, y_train_pred))
print("R2 test:", metrics.r2_score(y_test, y_test_pred))

print("Rmse training:", np.sqrt(metrics.mean_squared_error(y_train, y_train_pred)))
print("Rmse: testing", np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))

est = pipe.steps[-1][-1]
weights = pd.DataFrame({"feature": X.columns, "coef": est.coef_})
weights["abs"] = np.abs(weights["coef"])
weights = weights.sort_values("abs", ascending = False)
weights

R2 training: 0.9422562432501826
R2 test: 0.8756737399402362
Rmse training: 0.0919903782017158
Rmse: testing 0.15328521373355525


,feature,coef,abs
110,RoofMatl_CompShg,0.093301,0.093301
114,RoofMatl_Tar&Grv,0.062909,0.062909
3,OverallQual,0.061513,0.061513
15,GrLivArea,0.059623,0.059623
32,PoolArea,0.059227,0.059227
116,RoofMatl_WdShngl,0.049640,0.049640
236,PoolQC_None,0.045813,0.045813
12,1stFlrSF,0.038872,0.038872
13,2ndFlrSF,0.036688,0.036688
4,OverallCond,0.033106,0.033106


In [35]:
# dependent variable/target varible/label
label = "SalesPrice"

# independepent variables/features/predictors
X = df.drop(columns=[label])


# vector for target variable
y = np.log(df[label])

# one hot encoding to conver categorical features into numeric
# drop_first: remove first value of a categorical feature ... it is redundant
X = pd.get_dummies(X, drop_first=True)

# Creating trainnig and test sets
# test_size is 30% of the whole
# random_state: to reprduce the same combination of training and test records
X_train, X_test, y_train, y_test = model_selection.train_test_split(X.values,                       
                                    y, test_size = 0.3, random_state = 1)


pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.ElasticNet(alpha=0.01, l1_ratio=0.5))
])

pipe.fit(X_train, y_train)

# prediction on training and test data
y_train_pred = pipe.predict(X_train)
y_test_pred = pipe.predict(X_test)

print("R2 training:", metrics.r2_score(y_train, y_train_pred))
print("R2 test:", metrics.r2_score(y_test, y_test_pred))

print("Rmse training:", np.sqrt(metrics.mean_squared_error(y_train, y_train_pred)))
print("Rmse: testing", np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))

est = pipe.steps[-1][-1]
weights = pd.DataFrame({"feature": X.columns, "coef": est.coef_})
weights["abs"] = np.abs(weights["coef"])
weights = weights.sort_values("abs", ascending = False)
weights

R2 training: 0.9119521556819916
R2 test: 0.8832497152982852
Rmse training: 0.11359239347908372
Rmse: testing 0.1485414992986777


,feature,coef,abs
15,GrLivArea,0.091909,0.091909
3,OverallQual,0.088640,0.088640
25,GarageCars,0.049258,0.049258
5,YearBuilt,0.035001,0.035001
4,OverallCond,0.028759,0.028759
70,Neighborhood_NridgHt,0.026675,0.026675
60,Neighborhood_Crawfor,0.026182,0.026182
39,MSZoning_RM,-0.021363,0.021363
90,Condition2_PosN,-0.020887,0.020887
16,BsmtFullBath,0.019741,0.019741


In [66]:
# dependent variable/target varible/label
label = "SalesPrice"

# independepent variables/features/predictors
X = df.drop(columns=[label])


# vector for target variable
y = np.log(df[label])

# one hot encoding to conver categorical features into numeric
# drop_first: remove first value of a categorical feature ... it is redundant
X = pd.get_dummies(X, drop_first=True)

# Creating trainnig and test sets
# test_size is 30% of the whole
# random_state: to reprduce the same combination of training and test records
X_train, X_test, y_train, y_test = model_selection.train_test_split(X.values,                       
                                    y, test_size = 0.3, random_state = 14234200)


pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.SGDRegressor(alpha=0.6, l1_ratio=1.0
                                      , random_state=1, max_iter=1000))
])

pipe.fit(X_train, y_train)

# prediction on training and test data
y_train_pred = pipe.predict(X_train)
y_test_pred = pipe.predict(X_test)

print("R2 training:", metrics.r2_score(y_train, y_train_pred))
print("R2 test:", metrics.r2_score(y_test, y_test_pred))

print("Rmse training:", np.sqrt(metrics.mean_squared_error(y_train, y_train_pred)))
print("Rmse: testing", np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))

est = pipe.steps[-1][-1]
weights = pd.DataFrame({"feature": X.columns, "coef": est.coef_})
weights["abs"] = np.abs(weights["coef"])
weights = weights.sort_values("abs", ascending = False)
weights

R2 training: 0.915552842519412
R2 test: 0.8581107360005259
Rmse training: 0.11933058264162974
Rmse: testing 0.13992569752082593


,feature,coef,abs
3,OverallQual,0.040529,0.040529
15,GrLivArea,0.031641,0.031641
12,1stFlrSF,0.025495,0.025495
90,Condition2_PosN,-0.023782,0.023782
25,GarageCars,0.023692,0.023692
22,TotRmsAbvGrd,0.023491,0.023491
235,PoolQC_Gd,-0.022272,0.022272
4,OverallCond,0.020845,0.020845
70,Neighborhood_NridgHt,0.020147,0.020147
6,YearRemodAdd,0.018839,0.018839


In [71]:
# dependent variable/target varible/label
label = "SalesPrice"

# independepent variables/features/predictors
X = df.drop(columns=[label])


# vector for target variable
y = np.log(df[label])

# one hot encoding to conver categorical features into numeric
# drop_first: remove first value of a categorical feature ... it is redundant
X = pd.get_dummies(X, drop_first=True)

pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.SGDRegressor(alpha=0.6, l1_ratio=1.0
                                      , random_state=1, max_iter=1000))
])

scores = model_selection.cross_val_score(pipe, X.values, y, cv = 5, scoring="r2")
np.mean(scores), scores

(0.8611492305867117,
 array([0.89411488, 0.86016541, 0.87371732, 0.88079949, 0.79694904]))

In [76]:
10 ** np.linspace(-2, 1, 10) 

array([ 0.01      ,  0.02154435,  0.04641589,  0.1       ,  0.21544347,
        0.46415888,  1.        ,  2.15443469,  4.64158883, 10.        ])

In [78]:
param_grid = {
    "est__alpha": 10 ** np.linspace(-2, 1, 10),
    "est__l1_ratio": np.linspace(0, 1, 10)
}

gsearch = model_selection.GridSearchCV(pipe, param_grid, cv = 5)
gsearch.fit(X.values, y)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('poly', PolynomialFeatures(degree=1, include_bias=False, interaction_only=False)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('est', SGDRegressor(alpha=0.6, average=False, early_stopping=False, epsilon=0.1,
       eta0=0.01, fit_intercept=True, l1_ratio=1.0,
     ...ndom_state=1, shuffle=True, tol=None, validation_fraction=0.1,
       verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'est__alpha': array([ 0.01   ,  0.02154,  0.04642,  0.1    ,  0.21544,  0.46416,
        1.     ,  2.15443,  4.64159, 10.     ]), 'est__l1_ratio': array([0.     , 0.11111, 0.22222, 0.33333, 0.44444, 0.55556, 0.66667,
       0.77778, 0.88889, 1.     ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [79]:
gsearch.best_score_

0.8620195490513789

In [80]:
gsearch.best_params_

{'est__alpha': 0.46415888336127775, 'est__l1_ratio': 0.0}

In [81]:
import pickle

In [83]:
with open("model.pkl", "wb") as f:
    pickle.dump(gsearch, f)

In [84]:
!ls -l

total 1552
-rw-r--r--  1 abasar  staff   14118 Dec 10 15:00 Day 1 - numpy.ipynb
-rw-r--r--  1 abasar  staff  228303 Dec 10 12:51 Day 1 - pandas.ipynb
-rw-r--r--  1 abasar  staff   25714 Dec 10 12:03 Day 1 - python.ipynb
-rw-r--r--  1 abasar  staff  213137 Dec 10 16:49 Day 1 - visualization.ipynb
-rw-r--r--  1 abasar  staff   23534 Dec 10 14:40 Day 1 Top 10 movies.ipynb
-rw-r--r--  1 abasar  staff   43709 Dec 11 12:39 Day 2 - regression.ipynb
-rw-r--r--  1 abasar  staff  114807 Dec 11 15:54 Untitled.ipynb
drwxr-xr-x  3 abasar  staff      96 Dec 10 11:58 __pycache__
-rw-r--r--@ 1 abasar  staff      82 Dec 10 11:57 app.py
-rw-r--r--  1 abasar  staff   32280 Dec 11 15:54 model.pkl
-rw-r--r--  1 abasar  staff   47419 Dec 10 16:47 plot.jpg
-rw-r--r--@ 1 abasar  staff   23983 Dec 10 16:48 plot.pdf
drwxr-xr-x  4 abasar  staff     128 Dec 10 21:37 src
-rw-r--r--@ 1 abasar  staff     295 Dec 10 11:58 util.py


In [85]:
with open("model.pkl", "rb") as f:
    model = pickle.load(f)

In [87]:
y_pred = model.predict(X)
y_pred

array([12.23604675, 12.13730473, 12.29455154, ..., 12.45642426,
       11.88332789, 11.90509137])

In [90]:
input_record = X.values[0,:]

In [94]:
input_record = input_record.reshape((1,-1))
input_record.shape

(1, 258)

In [96]:
np.exp(model.predict(input_record))

array([206085.5705559])

In [100]:
df.SalesPrice[0]

208500.0

In [101]:
_, pvals = feature_selection.f_regression(X.values, y)

In [106]:
sig = pd.DataFrame({"feature": X.columns, "pval": pvals})
sig = sig.sort_values("pval", ascending=True)
sig = sig.iloc[:10, :]
sig

,feature,pval
3,OverallQual,0.000000e+00
15,GrLivArea,3.060209e-216
25,GarageCars,3.093756e-199
26,GarageArea,1.106255e-176
11,TotalBsmtSF,7.534551e-151
12,1stFlrSF,1.074698e-141
151,ExterQual_TA,4.749723e-141
18,FullBath,2.118958e-140
5,YearBuilt,1.103567e-135
24,GarageYrBlt,7.405437e-126


In [109]:
# dependent variable/target varible/label
label = "SalesPrice"

# independepent variables/features/predictors
X = df.drop(columns=[label])


# vector for target variable
y = np.log(df[label])

# one hot encoding to conver categorical features into numeric
# drop_first: remove first value of a categorical feature ... it is redundant
X = pd.get_dummies(X, drop_first=True)

# Creating trainnig and test sets
# test_size is 30% of the whole
# random_state: to reprduce the same combination of training and test records
X_train, X_test, y_train, y_test = model_selection.train_test_split(X.values,                       
                                    y, test_size = 0.3, random_state = 1)


pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("pca", decomposition.PCA(n_components=0.9)),
    ("est", linear_model.LinearRegression())
])

pipe.fit(X_train, y_train)

# prediction on training and test data
y_train_pred = pipe.predict(X_train)
y_test_pred = pipe.predict(X_test)

print("R2 training:", metrics.r2_score(y_train, y_train_pred))
print("R2 test:", metrics.r2_score(y_test, y_test_pred))

print("Rmse training:", np.sqrt(metrics.mean_squared_error(y_train, y_train_pred)))
print("Rmse: testing", np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))


R2 training: 0.9067241383478821
R2 test: 0.8521492652455901
Rmse training: 0.11691615425115887
Rmse: testing 0.1671593110626173
